# APLICACIONES EN CIENCIAS DE COMPUTACION

## Laboratorio 3:  Busqueda con Informacion (Busqueda codiciosa, A* y heurísticas)

Indicaciones previas:
- Se recomienda utilizar Jupyter Notebook para la resolución del presente cuadernillo.
- Las respuestas deben tener un buen fundamento teórico, se realizarán descuentos en el puntaje a respuestas que no contesten a lo solicitado
- Cualquier indicio de plagio resultará en la anulación de la prueba.

La tarea de este laboratorio consiste en comparar métodos de busqueda a ciegas y busqueda con información A* para buscar caminos en un laberinto.<br>La clase MazeSearchProblem necesita ser completada (metodos actions y result).<br>Deberá implementarse 4 heurísticas para A* (euclidean_dist:distancia en linea recta, manhattan_dist:distancia Manhattan, chebyshev_dist:distancia Chebyshev, lab03_dist: distancia implementada para el laboratorio 3).<br>Después de implementar lo anteriormente mencionado, realice las pruebas con el laberinto the_maze.txt.<br>Al final de este notebook se encuentran las preguntas que serán evaluadas en este laboratorio. 

**Usted deberá completar el código en las secciones indicadas con "COMPLETAR"**


### Clase <b>Maze</b>
La clase Maze es representada por una matriz (grid) de espacios, los cuales pueden estar libres o bloqueados. Cada celda del grid puede tener uno de los siguientes carateres:

    '#': Indica una celda con obstaculo  (impasable) 
    '~': Indica una celda con agua (pasable, con costo 6)
    '=': Indica una celda con arena (pasable, con costo 3)
    ' ': Indica una celda vacia (pasable con costo 1)
    'S': Indica la celda de inicio (Start)
    'E': Indica la celda de salida ( End )

In [1]:
class Maze:

    def __init__(self, grid):
        """ Construye el maze a partir del grid ingresado
            grid: debe ser una matriz (lista de listas), ejemplo [['#','S',' '],['#',' ','E']]  """
        self.grid = grid
        self.numRows = len(grid)
        self.numCols = len(grid[0])
        for i in range(self.numRows):
            for j in range(self.numCols):
                if len(grid[i]) != self.numCols:
                    raise "Grid no es Rectangular"
                if grid[i][j] == 'S':
                    self.startCell = (i,j)
                if grid[i][j] == 'E':
                    self.exitCell= (i,j)
        if self.exitCell == None:
            raise "No hay celda de Inicio"
        if self.startCell == None:
            raise "No hay celda de salida"
   
    def isPassable(self, row, col):
        """ Retorna true si la celda (row,col) es pasable  (' ' o '~' o '=') """
        return self.isWater(row, col) or isSAnd(row,col) or self.isClear(row, col)    
    
    def isBlocked(self, row,col):
        """ Retorna true si la celda (row,col) tiene obstaculo ('#') """
        return self.grid[row][col] == '#'   
    
    def isClear(self,row,col):
        return self.grid[row][col] == ' '
    
    def isSand(self, row,col):
        """ Retorna true si la celda (row,col) tiene obstaculo ('#') """
        return self.grid[row][col] == '='  
    
    def isWater(self, row, col):
        return self.grid[row][col] == '~' 
        
    def getNumRows(self):
        """ Retorna el numero de filas en el maze """
        return self.numRows
  
    def getNumCols(self):
        """ Retorna el numero de columnas en el maze """
        return self.numCols  
   
    def getStartCell(self):
        """ Retorna la posicion (row,col) de la celda de inicio """
        return self.startCell
  
    def getExitCell(self):
        """ Retorna la posicion (row,col) de la celda de salida """
        return self.exitCell

    def __getAsciiString(self):
        """ Retorna el string de vizualizacion del maze """
        lines = []
        headerLine = ' ' + ('-' * (self.numCols)) + ' '
        lines.append(headerLine)
        for row in self.grid:
            rowLine = '|' + ''.join(row) + '|'
            lines.append(rowLine)
        lines.append(headerLine)
        return '\n'.join(lines)

    def __str__(self):
        return self.__getAsciiString()

### Funcion para cargar una laberinto de archivo de disco

In [2]:
def readMazeFromFile(file):
    """ Lee un archivo que contiene un laberinto y retorna una instancia de Maze con dicho laberinto"""
    fin = open(file)
    lines = fin.read().splitlines()
    grid = []
    for line in lines: 
        grid.append(list(line))
    return Maze(grid)

### Clase <b>SearchProblem</b>

Esta es una clase abstracta para definir problemas de busqueda. Se debe hacer subclases que implementen los metodos de las acciones, resultados, test de objetivo y el costo de camino. Entonces se puede instanciar las subclases y resolverlos con varias funciones de busqueda.

In [3]:
class SearchProblem(object):
    def __init__(self, initial, goal=None):
        """Este constructor especifica el estado inicial y posiblemente el estado(s) objetivo(s),
        La subclase puede añadir mas argumentos."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones que pueden ser ejecutadas en el estado dado.
        El resultado es tipicamente una lista."""
        raise NotImplementedError

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada en el estado state.
        La accion debe ser alguna de self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Retorna True si el estado pasado satisface el objetivo."""
        raise NotImplementedError

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con 
        la accion action, asumiendo un costo c para llegar hasta state1. 
        El metodo por defecto cuesta 1 para cada paso en el camino."""
        return c + 1

###  <b> Clase MazeSearchProblem </b>  
Esta es una subclase de SearchProblem que implementa concretamente el problema de busqueda en laberinto. El constructor recibe el laberinto en un objeto maze. Cada estado es codificado como una tupla (row,col) representando la posicion de una celda del grid. El estado inicial es la celda de inicio y el único estado objetivo es la celda de salida.
Se necesita completar Actions (acciones legales para un estado dado) y result (que hacen las acciones).

In [4]:
class MazeSearchProblem(SearchProblem):
    def __init__(self, maze):
        """El constructor recibe el maze"""
        self.maze = maze
        self.initial = maze.getStartCell()
        self.goal = maze.getExitCell()
        self.numNodesExpanded = 0        
        self.expandedNodeSet = {}   
        
    def __isValidState(self,state):
        """ Retorna true si el estado dado corresponde a una celda no bloqueada valida """
        row,col = state
        if row < 0 or row >= self.maze.getNumRows():
            return False
        if col < 0 or col >= self.maze.getNumCols():
            return False
        return not self.maze.isBlocked(row,col)        

    def actions(self, state):
        """Retorna las acciones legales desde la celda actual """
        row,col = state
        acciones = []
        if self.__isValidState((row-1, col)):
            acciones.append('N')
            
        #########################################################################################################
            
                                                        #COMPLETAR
        if self.__isValidState((row+1, col)):
            acciones.append('S') 
            
        if self.__isValidState((row, col-1)):
            acciones.append('W') 

        if self.__isValidState((row, col+1)):
            acciones.append('E')

        #########################################################################################################
        
        return acciones
    
    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada desde la celda actual.
        La accion debe ser alguna de self.actions(state)"""  
        row,col = state
        newState = ()
        if action == 'N':
            newState = (row-1, col)
        
        #########################################################################################################
            
                                                        #COMPLETAR
        if action == 'S':
            newState = (row+1, col)
            
        if action == 'W':
            newState = (row, col-1)

        if action == 'E':
            newState = (row, col+1)
        
        #########################################################################################################
        
        return newState
    
        
    def goal_test(self, state):
        """Retorna True si state es self.goal"""
        return (self.goal == state) 

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con la accion action 
        El costo del camino para llegar a state1 es c. El costo de la accion sale de self.maze """
        row, col = state2
        
        if self.maze.isClear(row, col):
            actionCost =  1 # COMPLETAR COSTO
        elif self.maze.isWater(row, col):    
            actionCost =  6 # COMPLETAR COSTO
        elif self.maze.isSand(row, col):    
            actionCost =  3 # COMPLETAR COSTO
        elif state2 == self.maze.getStartCell() or state2 == self.maze.getExitCell():
            actionCost = 1
        else:
            raise "El costo de una celda bloqueada no esta definido" 
        return c + actionCost


### Clase <b>Node</b>

Estructura de datos para almacenar la información de un nodo en un <b>arbol de busqueda</b>. Contiene información del nodo padre y el estado que representa el nodo. Tambien incluye la acción que nos llevo al presente nodo y el costo total del camino desde el nodo raíz hasta este nodo.

In [5]:
# ESTA CELDA NO NECESITA SER MODIFICADA
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        "Crea un nodo de arbol de busqueda, derivado del nodo parent y accion action"
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        "Devuelve los nodos alcanzables en un paso a partir de este nodo."
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next = problem.result(self.state, action)
        return Node(next, self, action,
                    problem.path_cost(self.path_cost, self.state, action, next))

    def solution(self):
        "Retorna la secuencia de acciones para ir de la raiz a este nodo."
        return [node.action for node in self.path()[1:]]

    def path(self):
        "Retorna una lista de nodos formando un camino de la raiz a este nodo."
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def __lt__(self, node):
        return self.state < node.state
    
    def __eq__(self, other): 
        "Este metodo se ejecuta cuando se compara nodos. Devuelve True cuando los estados son iguales"
        return isinstance(other, Node) and self.state == other.state
    
    def __repr__(self):
        return "<Node {}>".format(self.state)
    
    def __hash__(self):
        return hash(self.state)

### <b> Frontera tipo cola FIFO (first-in first out) para BFS</b> 

In [6]:
# ESTA CELDA NO NECESITA SER MODIFICADA
from collections import deque

class FIFOQueue(deque):
    """Una cola First-In-First-Out"""
    def pop(self):
        return self.popleft()

### <b> Frontera tipo cola de prioridad ordenada por una funcion de costo (para best_first_graph_search y A*)</b> 

In [7]:
# ESTA CELDA NO NECESITA SER MODIFICADA
import heapq
class FrontierPQ:
    "Una Frontera ordenada por una funcion de costo (Priority Queue)"
    
    def __init__(self, initial, costfn=lambda node: node.path_cost):
        "Inicializa la Frontera con un nodo inicial y una funcion de costo especificada (por defecto es el costo de camino)."
        self.heap   = []
        self.states = {}
        self.costfn = costfn
        self.add(initial)
    
    def add(self, node):
        "Agrega un nodo a la frontera."
        cost = self.costfn(node)
        heapq.heappush(self.heap, (cost, node))
        self.states[node.state] = node
        
    def pop(self):
        "Remueve y retorna el nodo con minimo costo."
        (cost, node) = heapq.heappop(self.heap)
        self.states.pop(node.state, None) # remove state
        return node
    
    def replace(self, node):
        "node reemplaza al nodo de la Fontera que tiene el mismo estado que node."
        if node.state not in self:
            raise ValueError('{} no tiene nada que reemplazar'.format(node.state))
        for (i, (cost, old_node)) in enumerate(self.heap):
            if old_node.state == node.state:
                self.heap[i] = (self.costfn(node), node)
                heapq._siftdown(self.heap, 0, i)
                return

    def __contains__(self, state): return state in self.states
    
    def __len__(self): return len(self.heap)

### <b>Algoritmo general de búsqueda con memoria de nodos expandidos (Graph Search)</b>

Algoritmo de general de busqueda ciega con memoria de estados visitados. El argumento frontier debe ser una cola vacia. Si la frontera es tipo FIFO hace busqueda en amplitud (BFS), si la frontera es una pila hará busqueda en profundidad (DFS)

In [8]:
# ESTA CELDA NO NECESITA SER MODIFICADA
def graph_search(problem, frontier):
    frontier.append(Node(problem.initial))
    explored = set()     # memoria de estados visitados
    visited_nodes = []   # almacena nodos visitados durante la busqueda
    while frontier:
        node = frontier.pop()
        visited_nodes.append(node)
        if problem.goal_test(node.state):
            return node, visited_nodes
        explored.add(node.state)
        
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None

### <b> Algoritmo Best-First-Graph-Search </b> 
Algoritmo general de busqueda con información. La frontera es una cola de prioridad ordenada por la funcion de evaluacion f 

In [9]:
# ESTA CELDA NO NECESITA SER MODIFICADA
def best_first_graph_search(problem, f):
    """Busca el objetivo expandiendo el nodo de la frontera con el menor valor de la funcion f. Memoriza estados visitados
    Antes de llamar a este algoritmo hay que especificar La funcion f(node). Si f es node.depth tenemos Busqueda en Amplitud; 
    si f es node.path_cost tenemos Busqueda  de Costo Uniforme. Si f es una heurística tenemos Busqueda Voraz;
    Si f es node.path_cost + heuristica(node) tenemos A* """

    frontier = FrontierPQ( Node(problem.initial), f )  # frontera tipo cola de prioridad ordenada por f
    explored = set()     # memoria de estados visitados
    visited_nodes = []   # almacena nodos visitados durante la busqueda
    while frontier:
        node = frontier.pop()
        visited_nodes.append(node)        
        if problem.goal_test(node.state):
            return node, visited_nodes
        explored.add(node.state)
        for action in problem.actions(node.state):
            child = node.child_node(problem, action)
            if child.state not in explored and child.state not in frontier:
                frontier.add(child)
            elif child.state in frontier:
                incumbent = frontier.states[child.state] 
                if f(child) < f(incumbent):
                    frontier.replace(child)

### <b> Algoritmo A* </b> 
A* es un caso especial de best_first_graph_search con f = path_cost + heuristic

In [10]:
# ESTA CELDA NO NECESITA SER MODIFICADA
def astar_search(problem, heuristic):
    f = lambda node: node.path_cost + heuristic(node, problem)
    return best_first_graph_search(problem, f)

def nullheuristic(node, problem):   # heurística nula (A* se convierte en busqueda de costo uniforme)
    return 0

### <b> Heurísticas para A* </b> 
Se debe implementar las heurísticas abajo para A* 

In [11]:
# DEBE COMPLETAR EL CÓDIGO DE CADA HEURÍSTICA DE ACUERDO A LA DESCRIPCIÓN DE CADA UNA

import math

def euclidean_dist(node, problem):
    "Distancia en linea recta desde la celda de node hasta la celda Objetivo (problem.goal)"
    ## COMPLETAR
    
    (sx , sy) = node.state     #estado actual (posicion de fila y columna)
    (gx , gy) = problem.goal   #estado objetivo (fila y columna)
    return ((gx-sx)**2+(gy-sy)**2)**0.5
    
     
def manhattan_dist(node, problem):
    "Distancia Manhattan (o city block) desde la celda de node hasta la celda Objetivo (problem.goal)"
    ## COMPLETAR
    
    (sx , sy) = node.state     #estado actual (posicion de fila y columna)
    (gx , gy) = problem.goal   #estado objetivo (fila y columna)
    return abs(gx-sx) + abs(gy-sy)
    
    
def chebyshev_dist(node, problem):
    "Distancia de chebyshev (https://chris3606.github.io/GoRogue/articles/grid_components/measuring-distance.html#chebyshev-distance) desde la celda de node hasta la celda Objetivo (problem.goal)"
    ## COMPLETAR
    (sx , sy) = node.state     #estado actual (posicion de fila y columna)
    (gx , gy) = problem.goal   #estado objetivo (fila y columna)
    return max(gx-sx,gy-sy)
        
    
def lab03_dist(node, problem):
    "Es el valor máximo de evaluar la distancia Euclidiana, Manhattan y Chebyshev"
    ## COMPLETAR
    return max(max(manhattan_dist(node,problem),chebyshev_dist(node,problem)),euclidean_dist(node,problem))

### <b> Función para mostrar los resultados </b> 

In [12]:
# ESTA CELDA NO NECESITA SER MODIFICADA
def displayResults(maze, visitedNodes, solutionNodes):
    """ Muestra los resultados de busqueda en el maze.   """
    grid_copy = []
    for row in maze.grid:
        grid_copy.append([x for x in row]) 
    for node in visitedNodes:
        row,col = node.state
        ch = maze.grid[row][col]
        if ch != 'S' and ch != 'E': grid_copy[row][col] = 'o' 
    for node in solutionNodes:  
        row,col = node.state
        ch = maze.grid[row][col]
        if ch != 'S' and ch != 'E': grid_copy[row][col] = 'x'    
    maze_copy = Maze(grid_copy)
    print (maze_copy)
    print ("x - celdas en la solucion")
    print ("o - celdas visitadas durante la busqueda")
    print ("-------------------------------")

## <b> Experimentación con los algoritmos de Busqueda</b> 


In [13]:
""" Carga un laberinto de archivo en disco e instancia el problema de busqueda.   """
maze = readMazeFromFile('maze3.txt') 
p = MazeSearchProblem(maze)
print(maze)

 --------------------------- 
|#################~~~~~~~~~~|
|#################    =~   #|
|~~  S~~~~    ==   =~==~   #|
| ~~~~~ ~~~  ====  ==~=~~  #|
| ~   ===~~   #==   =~=~~  #|
|~~   ===~~  ###    =~     #|
|~~~  ===~~  ##   ###~     #|
|~=~    ~~        ## ~   ###|
| =~   ~  ##  == =~ =~   ###|
|~~    ==###  == =~      ###|
|##########   ==        ~###|
|~~~~~~~~         ~~~~##~   |
| ### # # ### ### ====##=   |
| # # # # #   # #   ==~~~=  |
| ### # # #   ###   =~~~==  |
| #   # # #   #     =##~E   |
| #   ### ### #     =##     |
|~~~~~~~~~~~~~~~~~~~~~~~~~~~|
 --------------------------- 


### Búsqueda en amplitud (BFS)

In [14]:
nsol, visited_nodes = graph_search(p, FIFOQueue())
print('Solucion BFS: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion BFS: ['S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'E']. Nodos visitados=339. Costo Solucion = 69
 --------------------------- 
|#################oooooooooo|
|#################ooooooooo#|
|ooooSooooooooooooooooooooo#|
|ooooxooooooooooooooooooooo#|
|ooooxoooooooo#oooooooooooo#|
|ooooxooooooo###ooooooooooo#|
|ooooxooooooo##ooo###oooooo#|
|ooooxxxxxxxxooooo##ooooo###|
|ooooooooo##xoooooooooooo###|
|oooooooo###xoooooooooooo###|
|##########oxoooooooooooo###|
|oooooooooooxxxxxxoooo##oooo|
|o###o#o#o###o###xoooo##ooo |
|o# #o#o#o#ooo# #xoooooooo  |
|o###o#o#o#ooo###xxxxxxxo=  |
|o#ooo#o#o#ooo#oooooo##xE   |
|o#ooo###o###o#oooooo##o    |
|~ooooooooooooooooooooo~~~~~|
 --------------------------- 
x - celdas en la solucion
o - celdas visitadas durante la busqueda
-------------------------------


### Búsqueda en profundidad (DFS)

In [15]:
nsol, visited_nodes = graph_search(p, [])
print('Solucion DFS: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion DFS: ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'S', 'S', 'E', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'S', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'N', 'N', 'W', 'W', 'W']. Nodos visitados=110. Costo Solucion = 367
 --------------------------- 
|#################~~~~~~~~~~|
|#################    =~   #|
|~~  Sxxxxxxxxxxxxxxxxxxxxx#|
| ~~~~~ ~~~  ====  ==~=~~ x#|
| ~   ===~~   #oxxxxxxxxxxx#|
|~~   ===~~  ###x   =~     #|
|~~~  ===~~  ## xx###~     #|
|~=~    ~~       x## ~   ###|
| =~   ~  ##  == xxxxxxxx###|
|~~    ==###  == =~     x###|
|##########xxxxxxxxxxxxxx###|
|xxxxxxxxxxx      ~~~~##~   |
|x### # # ##

### Búsqueda A* con heurística nula (UCS)

In [16]:
nsol, visited_nodes = astar_search(p, nullheuristic)
print('Solucion A* y heuristica nula (UCS): {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion A* y heuristica nula (UCS): ['S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'E', 'E', 'S', 'S', 'S', 'S', 'W', 'W']. Nodos visitados=322. Costo Solucion = 61
 --------------------------- 
|#################ooooooooo~|
|#################ooooooooo#|
|ooooSooooooooooooooooooooo#|
|ooooxooooooooooooooooooooo#|
|ooooxoooooooo#oooooooooooo#|
|ooooxooooooo###ooooooooooo#|
|ooooxooooooo##ooo###oooooo#|
|ooooxxxxxxxxxxxxo##ooooo###|
|ooooooooo##ooooxoooooooo###|
|oooooooo###ooooxoooooooo###|
|##########oooooxxxxxxxxx###|
|~~~~ooooooooooooooooo##xxxo|
| ###o#o#o###o###ooooo##ooxo|
| # #o#o#o#ooo# #ooooooo~oxo|
| ###o#o#o#ooo###oooooo~=oxo|
| #   #o#o#ooo#oooooo##~Exxo|
| #   ###o###o#oooooo##   o |
|~~~~~~ooooooooooooooo~~~~~~|
 --------------------------- 
x - celdas en la solucion
o - celdas visitadas durante la busqueda
-------------------------------


### Búsqueda A* con heurística euclidean_dist (Completar el código para mostrar los resultados de la búsqueda con heurística euclidean_dist)

In [17]:
# COMPLETAR
# Puede guiarse del código de prueba de BFS, DFS y UCS implementados anteriormente.
nsol, visited_nodes = astar_search(p, euclidean_dist)
print('Solucion A* y heuristica euclidean: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())


Solucion A* y heuristica euclidean: ['S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'E', 'S', 'E', 'S', 'S', 'S', 'W', 'W']. Nodos visitados=276. Costo Solucion = 61
 --------------------------- 
|#################oo~~~~~~~~|
|#################ooooo~   #|
|ooooSooooooooooooooooo~   #|
|ooooxooooooooooooooo~=~~  #|
|ooooxoooooooo#oooooooo~~o #|
|ooooxooooooo###ooooooooooo#|
|ooooxooooooo##ooo###oooooo#|
|ooooxxxxxxxxxxxxo##ooooo###|
|ooooooooo##ooooxoooooooo###|
|oooooooo###ooooxoooooooo###|
|##########oooooxxxxxxxxx###|
|~~~~~~~oooooooooooooo##xxoo|
| ### # #o###o###ooooo##oxx |
| # # # #o#ooo# #ooooooo~ox |
| ### # #o#ooo###oooooo~==x |
| #   # #o#ooo#oooooo##~Exx |
| #   ###o###o#oooooo##     |
|~~~~~~~~o~~~oooooooo~~~~~~~|
 --------------------------- 
x - celdas en la solucion
o - celdas visitadas durante la busqueda
-------------------------------


### Búsqueda A* con heurística manhattan_dist (Completar el código para mostrar los resultados de la búsqueda con heurística manhattan_dist)

In [18]:
# COMPLETAR
# Puede guiarse del código de prueba de BFS, DFS y UCS implementados anteriormente.
nsol, visited_nodes = astar_search(p, manhattan_dist)
print('Solucion A* y heuristica manhattan: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion A* y heuristica manhattan: ['S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'E', 'S', 'E', 'S', 'S', 'S', 'W', 'W']. Nodos visitados=268. Costo Solucion = 61
 --------------------------- 
|#################~~~~~~~~~~|
|#################ooooo~   #|
|ooooSoooooooooooooo~o=~   #|
|ooooxooooooooooooooo~=~~  #|
|ooooxoooooooo#oooooooo~~o #|
|ooooxooooooo###ooooooooooo#|
|ooooxooooooo##ooo###~ooooo#|
|ooooxxxxxxxxxxxxo##ooooo###|
|ooooooooo##ooooxoooooooo###|
|oooooooo###ooooxoooooooo###|
|##########oooooxxxxxxxxx###|
|~~~~~~~oooooooooooooo##xxo |
| ### # #o###o###ooooo##oxx |
| # # # #o#ooo# #ooooooo~ox |
| ### # #o#ooo###oooooo~==x |
| #   # #o#ooo#oooooo##~Exx |
| #   ###o###o#oooooo##     |
|~~~~~~~~~~~~o~oooooo~~~~~~~|
 --------------------------- 
x - celdas en la solucion
o - celdas visitadas durante la busqueda
-------------------------------


### Búsqueda A* con heurística chebyshev_dist (Completar el código para mostrar los resultados de la búsqueda con heurística chebyshev_dist)

In [19]:
# COMPLETAR
# Puede guiarse del código de prueba de BFS, DFS y UCS implementados anteriormente.
nsol, visited_nodes = astar_search(p, chebyshev_dist)
print('Solucion A* y heuristica chebyshev: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion A* y heuristica chebyshev: ['S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'E', 'S', 'E', 'S', 'S', 'S', 'W', 'W']. Nodos visitados=286. Costo Solucion = 61
 --------------------------- 
|#################ooo~~~~~~~|
|#################ooooo~   #|
|ooooSooooooooooooooooo~   #|
|ooooxoooooooooooooooo=~~  #|
|ooooxoooooooo#oooooooo~~oo#|
|ooooxooooooo###ooooooooooo#|
|ooooxooooooo##ooo###oooooo#|
|ooooxxxxxxxxxxxxo##ooooo###|
|ooooooooo##ooooxoooooooo###|
|oooooooo###ooooxoooooooo###|
|##########oooooxxxxxxxxx###|
|~~~~~~~oooooooooooooo##xxoo|
| ### # #o###o###ooooo##oxxo|
| # # # #o#ooo# #ooooooo~oxo|
| ### # #o#ooo###oooooo~==xo|
| #   # #o#ooo#oooooo##~Exxo|
| #   ###o###o#oooooo##  ooo|
|~~~~~~~~o~~~oooooooo~~~~~~~|
 --------------------------- 
x - celdas en la solucion
o - celdas visitadas durante la busqueda
-------------------------------


### Búsqueda A* con heurística lab03_dist (Completar el código para mostrar los resultados de la búsqueda con heurística lab03_dist)

In [20]:
# COMPLETAR
# Puede guiarse del código de prueba de BFS, DFS y UCS implementados anteriormente.
nsol, visited_nodes = astar_search(p, lab03_dist)
print('Solucion A* y heuristica chebyshev: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())


Solucion A* y heuristica chebyshev: ['S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'E', 'S', 'E', 'S', 'S', 'S', 'W', 'W']. Nodos visitados=268. Costo Solucion = 61
 --------------------------- 
|#################~~~~~~~~~~|
|#################ooooo~   #|
|ooooSoooooooooooooo~o=~   #|
|ooooxooooooooooooooo~=~~  #|
|ooooxoooooooo#oooooooo~~o #|
|ooooxooooooo###ooooooooooo#|
|ooooxooooooo##ooo###~ooooo#|
|ooooxxxxxxxxxxxxo##ooooo###|
|ooooooooo##ooooxoooooooo###|
|oooooooo###ooooxoooooooo###|
|##########oooooxxxxxxxxx###|
|~~~~~~~oooooooooooooo##xxo |
| ### # #o###o###ooooo##oxx |
| # # # #o#ooo# #ooooooo~ox |
| ### # #o#ooo###oooooo~==x |
| #   # #o#ooo#oooooo##~Exx |
| #   ###o###o#oooooo##     |
|~~~~~~~~~~~~o~oooooo~~~~~~~|
 --------------------------- 
x - celdas en la solucion
o - celdas visitadas durante la busqueda
-------------------------------


## Preguntas

<b>1) Completar el código faltante (Clases y Heurísticas). (3 ptos)<br>   
<b>2) En cuanto a los resultados obtenidos, ¿los algoritmos BFS o DFS encuentran soluciones óptimas?.<br> &nbsp; ¿Qué requisito necesita el problema para que alguno de estos algoritmos encuentre soluciones óptimas? (3 ptos) </b><br>  
<b>3) En cuanto a las soluciones encontradas por A*, ¿ellas serán siempre óptimas?.<br> &nbsp; ¿Qué necesita la heurística para garantizar soluciones óptimas?.<br> &nbsp; ¿Las heurísticas planteadas para el laboratorio cumplen con estos requisitos?. Explique. (4 ptos)</b><br>

<b>4) En cuanto al costo temporal, ¿qué método visita menos nodos?.<br> &nbsp; ¿Cómo podría esto beneficiar a casos reales en los que se emplee algoritmos A*? (3 ptos) </b><br>  
<b>5) ¿Existe relación de dominancia entre algún par de heurísticas implementadas (No considerar heurística nula)?. &nbsp; <br>¿Guarda alguna relación la solución de menor costo temporal y la heurística dominante? Explique. (3 ptos) </b><br>  
<b>6) ¿Explique cómo expande nodos cada método (UCS y las 4 heurísticas)?.<br>¿Coinciden las propiedades de una heurística con las zonas exploradas por cada una en las pruebas? Explique. (4 ptos)</b><br>  

 

Pregunta 2:

Las soluciones con los algoritmos BFS o DFS no son optimas debido a que existen caminos de menor costo que no son tomados en cuenta por estas. Esto tiene mucho sentido dado que las soluciones con dichos algoritmos no poseen mayor informacion que la de sus nodos vecinos.
Por otro lado, es importante mencionar, que el DFS no encuentra una solucion optima dado que retorna la primera solucion existente visitando los nodos en profundidad, teniendo la posibilidad de que exista otro camino mas corto para llegar al objetivo. Por otro lado las soluciones tampoco son optimas,  debido a que el costo por cada acción no es uniforme.


Pregunta 3:

Para poder determinar si las soluciones encontradas por A* son optimas debemos considerar las condiciones de optimalidad las cuales son admisibilidad y consistencia. En este caso si serán optimas dado que son admisibles y consistentes y es por ello que encontrarán siempre una solución. Esto sucede gracias a que en cada nodo se verifica que el costo dado por la funcion de evaluacion siempre es mayor a la heuristica.
Por otro lado, las heuristicas son consistentes dado que para cada nodo n y sucesor n' generado verifica h(n)<= c(n,a,n')+h(n'). Ademas, sabemos por teoria que toda heuristica consistente es tambien admisible.


Pregunta 4:

El método de A* con heurística de distancia Manhattan es el que visita menos nodos. Esto se da porque dicho metodo es el mas dominante respecto a las demas heuristicas. Nos podemos dar cuenta de esto al comparar las formulas para obtener el costo de cada metodo. Esto puede beneficiar en casos reales de forma muy significativa. Por ejemplo podriamos encontrar la salida de un laberinto visitando menos nodos y de esta forma llegar mas rapido hacia nuestro destino.

Pregunta 5:

Si existe relacion de dominancia entre las heuristicas implementadas. 
En primer lugar, al comparar el método euclidean_dist y chebyshev_dist nos percatamos que sqrt(pow(gx-sx,2)+pow(gy-sy,2)) >= max(gy-sy,gx-sx). Esto se debe a que en el método euclidean_dist se utiliza la suma de cuadrados entre 2 valores y luego a esta se le saca la raiz, pero esos 2 valores son mayor a 0. Por otro lado, en el método chebyshev solo hallamos el mayor de la diferencia de las coordenadas.Por lo tanto, el método euclidean_dist es dominante ante el método chebyshev. Es por ello que visita menos nodos.
Al comparar el método manhattan_dist con el euclidean_dist nos percatamos que abs(gx-sx)+abs(gy-sy) >= sqrt(pow(gx-sx,2)+pow(gy-sy,2)). Esto sucede gracias a que en el metodo manhattan_dist se suman en valor absoluto (gx-sx) y (gy-sy) sin elevarlos al cuadrado y sacarle la raiz. Es por ello que el metodo manhattan es mas dominante ante el metodo euclidean_dist y por lo tanto(por transitividad) ante el metodo chebysev_dist.
Finalmente al comparar el metodo lab03_dist con manhattan_dist nos percatamaos que el primero siempre sera menor igual dado que nos devolvera la solucion que mayor resultado a obtenido entre las 3 soluciones anteriores. En el peor de los casos puede ser igual a manhattan pero nunca mayor. Por lo tanto la mas dominante sigue siendo la de manhattan. abs(gx-sx)+abs(gy-sy)>= max(abs(gx-sx)+abs(gy-sy),max(gy-sy,gx-sx),sqrt(pow(gx-sx,2)+pow(gy-sy,2)).

Analisis de la solucion de menor costo temporal:
Podemos ver que si se esta cumpliendo que las soluciones de menor costo temporal son las dominantes. Esto se puede ver dado que el analisis temporal esta directamente relacionado con la cantidad de nodos visitados y nuestras heuristicas dominantes(manhattan y lab03) son las que menor cantidad de ellos presentan, comprobando de esta forma sus propiedades de dominancia.

Pregunta 6:

A continuacion procedere a explicar como se expande cada metodo y posteriormente procedere a explicar si las propiedades de las heuristica guarda relacion con lo descrito.

1.UCS:
El metodo de costo uniforme puede expander sus nodos como un tree search o como un graph search donde el ultimo evitara ir por caminos repetidos. En general este metodo desecha caminos costosos y se queda con los menos costosos para que de esta forma podamos obtener un algoritmo mas optimo. En este metodo es recomendable que los costos de las acciones deben ser mayor a cero debido a la caracterizacion de la complejidad de tiempo.

2.Busqueda A* con heuristica euclidean_dist:
Podemos observar en el grafico que se ha recorrido mayor cantidad de nodos que con manhattan_dist y lab03_dist. Por otro lado vemos que el camino que encontro fue resultado de muchas pruebas con otros nodos, topandose tambien con obstaculos.

3.Busqueda A* con heuristica manhattan_dist:
En este caso podemos ver como esta solucion visita muchos nodos pero es mas optima que las otras heuristicas dado que visita menos nodos para poder llegar al objetivo. Todo esto se puede visualizar en la grafica de la solucion mostrada en la ejecucion del codigo. Por otro lado vemos que en los niveles superiores busca todo sino solamente en la parte izquieda dado que en ese punto encontro la solucion

4.Busqueda A* con heuristica chebyshev_dist:
En este caso podemos ver como esta solucion visita muchos mas nodos que con las anteriores heuristicas esto se debe a que el algoritmo tuvo que analizar muchos nodos para poder encontrar el objetivo, topandose con algunos obstaculos.

5.Busqueda A* con heuristica lab03_dist:
En este caso podemos analizar que junto a manhattan_dist son las que menor nodos visitan. Esto se debe a que en realidad lab03_dist esta combinando las 3 heuristicas y nos esta devolviendo la que menor nodos visita, es decir manhattan. En el grafico que se muestra al ejecutar el codigo podemos ver como va expandiendo su busqueda de forma que logra encontrar al objetivo.


ANALISIS:
Si coinciden las propiedades  de las heurísticas con las zonas exploradas por cada una en las pruebas dado que como hemos analizado anteriormente las busquedas con las heuristicas manhattan_dist y lab03_dist son las que menor cantidad de nodos visitados han presentado, esto se da dado que ambas son las heuristicas dominantes y la propiedad nos dice que en las heuristicas dominantes visitan menos nodos cosa que podemos ver que se cumple en el analisis decrito.